## 필요 패키지 설치

허깅페이스의 트랜스포머스 라이브러리를 설치합니다.

In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-1hpnmr7e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-1hpnmr7e
  Resolved https://github.com/huggingface/accelerate to commit 162a82164e9bdcc01a173cbee43b686437aaead8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.27.0.dev0-py3-none-any.whl size=273010 sha256=d4475a36e9d172ee07d58c1bd05a1e6487807830c8335d41062b4e3a5f1d90db
  Stored in directory: /tmp/pip-ephem-wheel-cache-24frsmh5/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


데이터 셋을 다운받는기 위해 다음 명령을 실행해서 허깅페이스 `Datasets` 라이브러리를 설치합니다.

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


그리고 T5 모델의 토크나이저가 `sentencepiece`를 사용하므로 다음을 실행해서 설치합니다.

In [4]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


또 모델 평가를 위해 허깅페이스 `evaluate` 라이브러리와 BLEU 점수를 계산하기위해 `sacrebleu`를 설치합니다.

In [5]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [6]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.8 MB/s eta 0:00:00


## Data Set

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import json

with open("/content/drive/MyDrive/NLP 실습/1 22/hw_sample.json") as f:
  data = json.load(f)

In [9]:
data.keys()

dict_keys(['chunk-number', 'paragraph'])

In [10]:
len(data['paragraph'])

4

In [12]:
len(data['paragraph'][0]['sentences'])

838

In [13]:
# 데이터 셋을 pandas로 바꿔주기 위해 pandas를 임포드 합니다.
import pandas as pd

In [14]:
df = pd.DataFrame(data['paragraph'][0]['sentences'])

In [15]:
len(df)

838

In [16]:
df.head(2)

,src_sentence,tgt_sentence,src_lang,tgt_lang,src_paragraphs_id,src_word_count,spc_technical_label,spc_idiomatic_label,spc_proper_label,spc_double_label
0,교토에 있는 수많은 절은 조선의 불상과 불경 등을 얻기 위해 혈안이 되어 있었다.,The statues and sutras of Joseon were in high ...,ko,en,SI-2208191639P00639047R0101120,13,"절,불상,불경",None,"교토,조선",None
1,"745년에 처음 창건됐고, 1709년에 다시 세워진 세계 최대의 목조 건물이다.",It is the largest wooden structure in the worl...,ko,en,SI-2208191639P00639047R0101366,10,세계,None,None,None


In [17]:
from sklearn.model_selection import train_test_split
# train: 0.8, valid: 0.1, test: 0.1
# train: 0.9, valid: 0.05, test: 0.05

In [18]:
df.head(1)

,src_sentence,tgt_sentence,src_lang,tgt_lang,src_paragraphs_id,src_word_count,spc_technical_label,spc_idiomatic_label,spc_proper_label,spc_double_label
0,교토에 있는 수많은 절은 조선의 불상과 불경 등을 얻기 위해 혈안이 되어 있었다.,The statues and sutras of Joseon were in high ...,ko,en,SI-2208191639P00639047R0101120,13,"절,불상,불경",None,"교토,조선",None


In [19]:
X_train, X_test, y_train, y_test = train_test_split(df[['src_sentence']], df[['tgt_sentence']], \
                                                     test_size=0.2, random_state=42)

In [20]:
train = pd.concat([X_train, y_train], axis=1)

In [21]:
valid_test = pd.concat([X_test, y_test], axis=1) # test, valid

In [22]:
X_valid, X_test, y_valid, y_test = train_test_split(valid_test[['src_sentence']], valid_test[['tgt_sentence']], \
                                                     test_size=0.5, random_state=42)

In [23]:
valid = pd.concat([X_valid, y_valid], axis=1) # test, valid
test = pd.concat([X_test, y_test], axis=1) # test, valid

In [24]:
print(len(train))
print(len(valid))
print(len(test))

670
84
84


In [25]:
# 필요할 때 파일을 읽어 허깅페이스 데이터 셋을 만들어 활용하기 위해 정데한 데이터 셋을 저장합니다.
train.to_csv("train.csv", index=False)
valid.to_csv("valid.csv", index=False)
test.to_csv("test.csv", index=False)

필요할때 이 파일을 읽어 허깅페이스 데이터셋을 만들 수 있습니다.

아래처럼 데이터 셋과 데이터 셋의 위치를 정의한 사전을 `load_dataset`에 넘기면 됩니다.

In [26]:
data_files = {"train": "train.csv", "valid": "valid.csv", "test": "test.csv"}

In [40]:
from datasets import load_dataset

In [41]:
dataset =  load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

제대로 로딩되었는지 `dataset`을 확인해봅니다.

In [42]:
dataset

DatasetDict({
    train: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 670
    })
    valid: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 84
    })
    test: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 84
    })
})

`DatasetDict`에 `train`, `valid`, `test` 키로 1000 문장, 100 문장, 100 문장이 저장된 것을 확인할 수 있다.

이 데이터 셋에서 개별 샘플에 대한 접근은 `[split][feature][row num]` 형태로 가능하다.

In [45]:
# train 스플릿에서 영어 3개와 한국어 3개 샘플을 가져옵니다.
print(dataset['train']['src_sentence'][:3])
print(dataset['train']['tgt_sentence'][:3])

['러일전쟁의 결과 일본은 한국 외에도 사할린 남부, 요동반도, 만주 남부로 세력을 넓혀 제국주의 국가로서의 지위를 확립했다.', '초대 벚나무가 말라죽자 1949년에 2대 벚나무를 심었다.', '오진 천황은 한국에서 위대한 공훈을 올린 분이니, 한국 합병을 받들어 고하여 축하하는 것은 당연한 일이다.']
['As a result of the Russo-Japanese War, Japan established its status as an empire by extending its influence over southern Sakhalin, the Yodong Peninsula, and southern Manchuria in addition to Korea.', 'In 1949, a second cherry tree was planted after the original one wilted and died.', 'Since Emperor Ojin accomplished so much in Korea, it is only fitting that we commemorate the annexation of Korea in his honor.']


이때, `feature`와 `row num`은 순서를 바꿔서 사용할 수 도 있습니다.

In [46]:
print(dataset['train'][:3]['src_sentence'])
print(dataset['train'][:3]['tgt_sentence'])

['러일전쟁의 결과 일본은 한국 외에도 사할린 남부, 요동반도, 만주 남부로 세력을 넓혀 제국주의 국가로서의 지위를 확립했다.', '초대 벚나무가 말라죽자 1949년에 2대 벚나무를 심었다.', '오진 천황은 한국에서 위대한 공훈을 올린 분이니, 한국 합병을 받들어 고하여 축하하는 것은 당연한 일이다.']
['As a result of the Russo-Japanese War, Japan established its status as an empire by extending its influence over southern Sakhalin, the Yodong Peninsula, and southern Manchuria in addition to Korea.', 'In 1949, a second cherry tree was planted after the original one wilted and died.', 'Since Emperor Ojin accomplished so much in Korea, it is only fitting that we commemorate the annexation of Korea in his honor.']


데이터를 어떻게 조회하는지는 데이터 구성 방식에 따라 조금씩 다르므로 데이터 셋을 보고 몇번 해보면 금방 접근법을 알 수 있다.

## Hugging face

데이터 셋 준비를 마쳤으니 학습을 진행해 봅니다. 허깅페이스에서 제공하는 필요 클래스를 임포트 합니다.

먼저 사전학습된 모델을 사용하기 위한 클래스를 임포트 합니다. `AutoTokenizer`는 사전학습된 모델이 사용한 토크나이저를 읽기 위해 필요하며 `AutoModelForSeq2SeqLM`은 시퀀스 투 스퀀스 방식으로 작동하는 사전학습된 모델을 불러 올 때 마지막에 분류기 헤드를 붙여서 모델을 로딩하기 위해 사용합니다.

In [44]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

다음은 데이터 콜레이터를 임포트합니다. 데이터 단위 들을 적절히 정리해서 모델에 입력하는 작업이 필요합니다. 이 과정은 `DataCollatorForSeq2Seq`가 자동으로 처리하게 됩니다.

In [31]:
from transformers import DataCollatorForSeq2Seq

그리고 학습에 필요한 클래스를 임포트 합니다. 학습에 필요한 설정을 `Seq2SeqTrainingArguments`에 정의하고 실제 학습은 `Seq2SeqTrainer`로 하게 됩니다.
`Seq2SeqTrainer`는 `generate()`함수를 제공합니다.

In [32]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

데이터 셋을 로딩하는 함수와 번역 결과를 측정할 함수를 로딩합니다.

In [33]:
from datasets import load_dataset, load_metric

그외 필요한 라이브러리들을 임포트 합니다.

In [34]:
import numpy as np
import torch

허깅페이스에서 파이토치 기반 구현을 사용하므로 gpu가 있다면 `device`를 세팅합니다.

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

미리 학습된 모델의 체크포인트를 세팅합니다. 여기서 사용할 모델은 한국어와 영어에 미리 학습된 [KE-T5](https://github.com/AIRC-KETI/ke-t5)모델을 사용합니다. T5모델은 트랜스포머의 인코더, 디코더 구조를 모두 사용하는 모델로 번역기를 만들 때 사용할 수 있는 모델이다. 아래처럼 모델 체크 포인트와 T5 모델에 입력될 최대 토큰 길이를 설정합니다.

In [36]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

## Tokenizer

먼저 모델 체크 포인트를 사용하여 KE-T5 모델이 학습할때 함께 사용한 토크나이저를 불러옵니다.

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


토크나이저를 불러왔으니 현재 사용하는 데이터셋에서 샘플을 가져와 토크나이징을 진행해 봅니다.

In [47]:
dataset['train'][1]['src_sentence'], dataset['train'][1]['tgt_sentence']

('초대 벚나무가 말라죽자 1949년에 2대 벚나무를 심었다.',
 'In 1949, a second cherry tree was planted after the original one wilted and died.')

토크나이저에 각 문장을 입력하면 토큰화된 결과를 확인할 수 있습니다.

In [48]:
tokenized_sample_en = tokenizer(dataset['train'][1]['src_sentence'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [5749, 7, 63826, 3699, 22, 9254, 4463, 127, 38859, 1746, 51, 109, 7, 63826, 3699, 21, 1070, 829, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
tokenized_sample_ko = tokenizer(dataset['train'][1]['tgt_sentence'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_ko

{'input_ids': [215, 38859, 4, 16, 870, 60748, 12646, 62, 46605, 272, 5, 4635, 165, 4248, 3245, 2830, 13, 3566, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

문장에 토큰으로 쪼개지고 각 토큰이 숫자로 변환된 것을 볼 수 있습니다. 이렇게 숫자화된 토큰을 `input_ids`로 반환하고 추가로 트랜스포머 인코더, 디코더에 쓰일 패딩 마스크도 함께 `attention_mask`로 돌려줍니다. 마스크가 모두 1인 이유는 샘플이 하나밖에 없기 때문입니다.

샘플 몇개를 더 실험해보겠습니다.

In [50]:
tokenizer(dataset['train'][:3]['tgt_sentence'],
          max_length=max_token_length,
          padding=True, truncation=True)

{'input_ids': [[806, 16, 2924, 14, 5, 7685, 8, 7432, 28, 46789, 15543, 13618, 3834, 4, 6673, 8664, 173, 7260, 64, 93, 46761, 81, 40923, 173, 7158, 246, 13063, 388, 42901, 482, 476, 4, 5, 17316, 136, 5097, 48857, 4, 13, 13063, 3650, 38962, 17466, 20, 3407, 10, 5701, 3, 1], [215, 38859, 4, 16, 870, 60748, 12646, 62, 46605, 272, 5, 4635, 165, 4248, 3245, 2830, 13, 3566, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2396, 48394, 243, 774, 1042, 19040, 34544, 262, 581, 20, 5701, 4, 60, 43, 387, 45195, 38, 146, 58645, 5, 54428, 1387, 14, 5701, 20, 111, 9395, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 

샘플 3개 중 최대 길이를 갖는 샘플에 맞춰서 패딩되는 모습을 확인할 수 있습니다.

실제로 dataset['train'][1]['en']에 대해서 어떻게 토큰화 되었는지 확인해보겠습니다.

In [51]:
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ids,5749,7,63826,3699,22,9254,4463,127,38859,1746,51,109,7,63826,3699,21,1070,829,3,1
tokens,▁초대,▁,벚,나무,가,▁말라,죽,자,▁1949,년에,▁2,대,▁,벚,나무,를,▁심,었다,.,</s>


실제로 dataset['train'][1]['ko']에 대해서 어떻게 토큰화 되었는지 확인해보겠습니다.

In [52]:
pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ids,215,38859,4,16,870,60748,12646,62,46605,272,5,4635,165,4248,3245,2830,13,3566,3,1
tokens,▁In,▁1949,",",▁a,▁second,▁cherry,▁tree,▁was,▁planted,▁after,▁the,▁original,▁one,▁w,il,ted,▁and,▁died,.,</s>


KE-T5를 학습할때 학습된 규칙대로 토큰화가 진행됩니다. 영어에서 `rechargeable`은 `charge`, `able`으로 쪼개진 것을 볼 수 있고, 한국어에서 `광마우스`은 `광`, `마우스`로 쪼개진것을 볼 수 있습니다. 토큰 앞에 _표시는 이 토큰 앞에는 공백이 있어야 한다는 의미입니다. 그리고 마지막에 엔드 토큰인 `</s>`가 항상 붙게 되는 것도 확인할 수 있습니다.

이제 앞서 csv파일로 부터 로딩한 `dataset`내의 문장을 모두 토크나이저를 사용해서 숫자로 바꾸는 작업을 해야 합니다. 즉 문자로된 문장을 숫자로 바꿔 특성화 해야 합니다. `dataset.map()`함수에 각 샘플을 토큰화 하는 함수를 만들어 전달하면 `map()`이 모든 샘플에 대해 전달받은 함수를 적용하게 되는데 함수는 아래와 같이 작성합니다.

In [53]:
def convert_examples_to_features(examples):
    model_inputs = tokenizer(examples['src_sentence'],
                             text_target=examples['tgt_sentence'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

`convert_examples_to_features()`가 하고 싶은 일은 `dataset`에 있는 샘플 문장을 정수로 바꿔줍니다. `convert_examples_to_features()`가 `dataset`에 적용될 때 넘겨 받는 `examples`는 다음과 같이 넘어 옵니다.

```python
examples= {'en':['sent1', 'sent2', ... , 'sent10'], # 이건 문장 10개짜리 리스트
           'ko':['sent1', 'sent2', ... , 'sent10']}
```

그리고 `remove_columns` 인자에 기존 특성 이름인 `en`, `ko`를 전달해서 기존 특성은 제거하게 한다. 이 특성이 있으면 이후 콜레이터가 샘플들을 미니 배치로 묶을 때 패딩처리를 못하게 된다.

In [54]:
dataset["train"].column_names

['src_sentence', 'tgt_sentence']

In [55]:
dataset

DatasetDict({
    train: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 670
    })
    valid: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 84
    })
    test: Dataset({
        features: ['src_sentence', 'tgt_sentence'],
        num_rows: 84
    })
})

In [56]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 # 이걸 쓰지 않으면 원 데이터 'en', 'ko'가 남아서 에러남
                                 remove_columns=dataset["train"].column_names
                                 )

Map:   0%|          | 0/670 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

`convert_examples_to_features()`이 `dataset`의 모든 샘플에 다 적용되고 나면 `tokenized_datasets`는 다음처럼 됩니다.

In [57]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 670
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 84
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 84
    })
})

기존에 있던 특성 `en`, `ko`는 사라졌고 `en`은 `input_ids`와 `attention_mask`로 `ko`는 `labels`로 바뀐것을 확인할 수 있습니다. 예를 들어 학습 세트에 10번 데이터를 보면 다음처럼 다 숫자라 바뀌게 된것이다.

In [58]:
tokenized_datasets['train'][10]

{'input_ids': [188,
  521,
  16078,
  15998,
  479,
  1495,
  56767,
  228,
  4,
  12663,
  11,
  2932,
  479,
  9494,
  4372,
  27,
  2161,
  29243,
  141,
  3,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [283,
  1321,
  246,
  5,
  1014,
  861,
  4,
  17271,
  60,
  4,
  13,
  2685,
  16,
  31511,
  575,
  474,
  92,
  10198,
  10,
  3876,
  3,
  1]}

토크나이저를 써서 숫자로 부터 토큰화 해보면 다음과 같습니다.

In [60]:
print( '원 데이터    :', dataset['train'][1]['src_sentence'] )
print( '처리 후 데이터:', tokenized_datasets['train'][1]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][1]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][1]['tgt_sentence'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][1]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][1]['labels'] )

원 데이터    : 초대 벚나무가 말라죽자 1949년에 2대 벚나무를 심었다.
처리 후 데이터: [5749, 7, 63826, 3699, 22, 9254, 4463, 127, 38859, 1746, 51, 109, 7, 63826, 3699, 21, 1070, 829, 3, 1]
토큰화       : ['▁초대', '▁', '벚', '나무', '가', '▁말라', '죽', '자', '▁1949', '년에', '▁2', '대', '▁', '벚', '나무', '를', '▁심', '었다', '.', '</s>']


원 데이터    : In 1949, a second cherry tree was planted after the original one wilted and died.
처리 후 데이터: ['▁In', '▁1949', ',', '▁a', '▁second', '▁cherry', '▁tree', '▁was', '▁planted', '▁after', '▁the', '▁original', '▁one', '▁w', 'il', 'ted', '▁and', '▁died', '.', '</s>']
토큰화       : [215, 38859, 4, 16, 870, 60748, 12646, 62, 46605, 272, 5, 4635, 165, 4248, 3245, 2830, 13, 3566, 3, 1]


데이터 특성화를 모두 마쳤으므로 이제 모델을 로딩하자. `AutoModelForSeq2SeqLM`를 사용해서 사전학습된 모델을 불러오면 사전학습된 T5모델 마지막에 파인튜닝할 수 있는 분류 헤드를 붙인 모델을 반환하게됩니다.

## Model

아래와 같이 모델을 로딩하고 모델 출력 시켜보면 T5 모델 레이어가 매우 길게 출력되는데 제일 마지막 부분에 다음과 같이 분류 헤드가 붙어 있는 것을 확인할 수 있습니다. 헤드를 보면 모델에서 출력하는 벡터는 768차원이고 이를 단어장 사이즈인 64128로 변환시키고 있는 것을 알 수 있습니다.

```
(lm_head): Linear(in_features=768, out_features=64128, bias=False)
```

이렇게 생성된 `model`은 인코더-디코더 구조를 가지는 트랜스포머이므로 이 모델을 포워딩 하려면 인코더 인풋과 디코더 인풋을 넣어줘야 합니다.


이때, 임의의 입력을 넣고 출력이 의도대로 나오는지 확인함으로써 모델이 제대로 작동 되는지 어떤 구조로 되어있는지 확인할 수 있습니다. 테스트를 하기위해 간단한 영어문장으로 테스트 해보겠습니다.



In [61]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [62]:
encoder_inputs = tokenizer(
    ["Studies have been shown that owning a dog is good for you"],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["개를 키우는 것이 건강에 좋다는 연구 결과가 있습니다."],
    return_tensors="pt"
)['input_ids'].to(device)


영어 문장은 인코더의 입력이 되고 한국어 문장은 디코더의 타겟이 됩니다. 아래처럼 모두 숫자로 변환되어 있습니다.

In [63]:
print( encoder_inputs )
print( decoder_targets )

tensor([[24611,    84,   166,  8135,    38,   847,    91,    16,  8146,    43,
           667,    40,   106,     1]], device='cuda:0')
tensor([[15833, 12236,   179, 16120, 28117,  1007,  3883,   327,     3,     1]],
       device='cuda:0')


이제 디코더 입력을 만들기위해 `model._shift_right`를 사용해 디코더 출력을 오른쪽으로 쉬프트 시킵니다.

In [64]:
decoder_inputs = model._shift_right(decoder_targets)

`decoder_inputs`와 `decoder_targets`이 어떻게 다른지 비교해보면

In [65]:
pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

,0,1,2,3,4,5,6,7,8,9
decoder target,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.,</s>
decoder input,<pad>,▁개를,▁키우는,▁것이,▁건강에,▁좋다는,▁연구,▁결과가,▁있습니다,.


위처럼 오른쪽으로 쉬프트된 디코더 입력은 `<pad>` 토큰이 추가되었다. 이렇게 출력으로 쓰이는 문장을 오른쪽으로 쉬프트시켜 티처포싱Teacher forcing을 진행하게 됩니다. 다음처럼 `model`에 인코더 입력, 디코더 입력, 디코더 타겟을 입력하고 포워드 시킵니다.

In [66]:
# forward pass
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

`model`의 `outputs`에는 다음과 같은 키가 있습니다.

In [67]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

손실함수 값을 다음처럼 확인할 수 있고 `grad_fn`이 있기 때문에 `output.loss`를 백워드 시킬 수 있다.

In [68]:
outputs.loss

tensor(87.8185, device='cuda:0', grad_fn=<NllLossBackward0>)

인코더의 마지막 상태는 (1, 14, 768)입니다. 각 숫자는 순서대로 샘플 수, 스탭 수, 모델 사이즈를 나타냅니다. 즉 인코더로 들어가는 14개 토큰이 각각 768차원 벡터로 인코딩되었습니다.

In [69]:
outputs['encoder_last_hidden_state'].shape

torch.Size([1, 14, 768])

`logit`은 디코더 입력 토큰 10개에 대한 그 다음 토큰 예측 10개를 담고있습니다. 샘플 한개에 대해서 10개 토큰에 대해서 64128개 단어에 대한 확률값이 들어 있습니다.

In [70]:
outputs['logits'].shape

torch.Size([1, 10, 64128])

`logit`에 `argmax`를 씌워서 토큰화시켜보면 다음과 같습니다.

In [71]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

['큐브', '큐브', '▁비일비재', '▁비일비재', '▁베네', '▁비일비재', '▁베네', '▁베네', '큐브', '큐브']

마지막 헤더가 학습이 되지 않았기 때문에 적절한 아웃풋이 나오지 않지만 입력과 출력의 텐서 모양을 보면 포워드 패스가 제대로 작동한다는 것을 알 수 있습니다.

## Collator

파이토치에서 모델을 학습시키기 위해서 `DataLoader`를 사용하게 되는데 이 데이터 로더의 역할은 for 루프를 돌면서 데이터 셋으로 부터 샘플을 배치 수만큼 가져오는 것입니다. 이때 샘플을 배치 수만큼 무작위로 가져와 어떤 식으로든 각 샘플을 짝맞춤해서 반환해야하는데 서로 크기가 다른 샘플들을 다루는 경우는 길이를 맞춘다든지 패딩을 한다든지 하는 추가 작업이 필요하게 됩니다. 이런 작업이 일어나는 곳이 `collate_fn`으로 지정되는 함수입니다.

시퀀스 투 시퀀스 모델을 학습시킬때 이런 콜레이터 함수가 하는 전형적인 역할은 입력 또는 출력 문자열을 패딩하고 조금 전 모델에서 알아봤듯이 디코더 타겟을 오른쪽으로 한칸 쉬프트 시켜서 디코더 입력으로 만드는 것 입니다. 이런 작업을 자동으로 처리해주는 클래스가 `DataCollatorForSeq2Seq`입니다.

우선 콜레이터를 만들기 위해서는 토크나이저와 모델을 넘겨야 합니다.

In [72]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

앞서 만들어논 `tokenized_datasets`에서 샘플 두개를 조회하면 다음처럼 전체 결과는 사전으로 리턴되며 사전의 각 키 아래에 여러 샘플들의 값이 리스트로 들어있게 됩니다.

In [73]:
# 각 항목아래 샘플들이 리스트 형태로 묶여 반환된다.
print(tokenized_datasets["train"][1:3])

{'input_ids': [[5749, 7, 63826, 3699, 22, 9254, 4463, 127, 38859, 1746, 51, 109, 7, 63826, 3699, 21, 1070, 829, 3, 1], [382, 214, 49999, 19, 6718, 11265, 1565, 2248, 11, 3914, 7288, 434, 4, 196, 42895, 56850, 446, 479, 4296, 74, 167, 12395, 4421, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[215, 38859, 4, 16, 870, 60748, 12646, 62, 46605, 272, 5, 4635, 165, 4248, 3245, 2830, 13, 3566, 3, 1], [2396, 48394, 243, 774, 1042, 19040, 34544, 262, 581, 20, 5701, 4, 60, 43, 387, 45195, 38, 146, 58645, 5, 54428, 1387, 14, 5701, 20, 111, 9395, 3, 1]]}


콜레이터에 샘플을 넘길 때는 개별 샘플이 사전으로 묶이는 형태가 되어야 되므로 아래처럼 한번 가공하게 된다.

In [74]:
# 콜레이터에는 샘플을 개별 {}로 넘겨야 합니다.
[tokenized_datasets["train"][i] for i in range(1, 3)]

[{'input_ids': [5749,
   7,
   63826,
   3699,
   22,
   9254,
   4463,
   127,
   38859,
   1746,
   51,
   109,
   7,
   63826,
   3699,
   21,
   1070,
   829,
   3,
   1],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  'labels': [215,
   38859,
   4,
   16,
   870,
   60748,
   12646,
   62,
   46605,
   272,
   5,
   4635,
   165,
   4248,
   3245,
   2830,
   13,
   3566,
   3,
   1]},
 {'input_ids': [382,
   214,
   49999,
   19,
   6718,
   11265,
   1565,
   2248,
   11,
   3914,
   7288,
   434,
   4,
   196,
   42895,
   56850,
   446,
   479,
   4296,
   74,
   167,
   12395,
   4421,
   3,
   1],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  'labels': [2396,
   48394,
   243,
   774,
   1042,
   19040,
   34544,
   262,
   581,
   20,
   5701,
   4,
   60,
 

위에 반환된 결과를 보면 각 샘플이 사전 {}으로 묶이고 샘플 하나에는 `input_ids`, `attention_mask`, ` labels`이 존재합니다. 각 샘플을 리스트로 묶어서 콜레이터에게 전달하고 반환되는 값을 확인해보겠습니다.

In [75]:
# 콜레이터를 돌리면 알아서 패딩하고 쉬프트 시킵니다.
batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

반환된 `batch`의 키를 확인해보면 `decoder_input_ids`가 생긴것을 확인할 수 있습니다.

In [76]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

`batch`의 각 키에 어떤 값들이 들어있는지 확인해봅니다.

In [77]:
batch

{'input_ids': tensor([[ 5749,     7, 63826,  3699,    22,  9254,  4463,   127, 38859,  1746,
            51,   109,     7, 63826,  3699,    21,  1070,   829,     3,     1,
             0,     0,     0,     0,     0],
        [  382,   214, 49999,    19,  6718, 11265,  1565,  2248,    11,  3914,
          7288,   434,     4,   196, 42895, 56850,   446,   479,  4296,    74,
           167, 12395,  4421,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'labels': tensor([[  215, 38859,     4,    16,   870, 60748, 12646,    62, 46605,   272,
             5,  4635,   165,  4248,  3245,  2830,    13,  3566,     3,     1,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ 2396, 48394,   243,   774,  1042, 19040, 34544,   262,   581,    20,
          5701,     4,    60,    43,   387, 45195,

출력된 batch를 정리하면 아래처럼 된다.


새로 생긴 `decoder_input_ids`는 앞에 0(<pad>)이 붙어 있는 것이 보이고 `label`에서 끝에 1이 사라져 `label`이 오른쪽으로 쉬프트된 것임을 알 수 있습니다. 그리고 또 두번째 샘플 `labels`에서 마지막에 -100 이 보인다. 이 값은 `label`이 패딩된 것을 나타내며 손실 함수값을 계산할 때 -100이 있는 위치는 손실을 계산하지 않게 됩니다. 이렇게 시퀀스 투 시퀀스 모델을 학습하기 위해 필요한 자잘한 작업을 콜레이터가 알아서 자동으로 처리합니다.
    
   
